In [1]:
import numpy as np
import plotly.graph_objects as go
import os
import gzip
import pickle as pkl

import games

# import TSPM
# import PM_DMED
# import cbp
# import bpm

In [2]:
def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                res = pkl.load(f)
                result[i] = res
            except EOFError:
                break
    return result

# direct = './results/benchmark_randcbp/AT/'
# path = os.path.join(direct, 'balanced_100_96_RandCBP_1_5.pkl.gz' )

# loadall_results(path, 100, 96)
# n_folds = 96
# horizon = 20000
# direct = './results/benchmark_randcbp/AT/'
# path = os.path.join(direct, 'balanced_20000_96_RandCBP_1_5.pkl.gz' )

# result = loadall_results(path, horizon, n_folds)
# print(result.shape)

# regret =  np.mean(result,0)

# Apple Tasting:


In [ ]:
n_cores = 16
n_folds = 100
horizon = 20000

game = games.apple_tasting(False)

colors = [   [250,0,0], [255,128,0], [255 , 0, 255], [0 , 0, 250], [0 , 128, 0] , [0,0,0] ] 
labels = [    'RandCBP', 'PM_DMEDc1',  'TSPM_R1', 'TSPM_R0', 'BPM_LEAST',  'CBP'  ] 

fig = go.Figure(    )

for color, label in list(zip(colors, labels))[::-1]:

    r,g,b = color

    if label == 'RandCBP':
        direct = './results/benchmark_randcbp/AT/'
        path = os.path.join(direct, 'imbalanced_20000_96_RandCBP_1_5.pkl.gz' )
        result = loadall_results(path, horizon, n_folds)

    elif label == 'PM_DMEDc1':
        result = np.load('./results/AT/easy_{}_{}_{}.npy'.format(20000,16, label) )
    else: 
        result = np.load('./results/AT/easy_{}_{}_{}.npy'.format(100000,n_folds, label) )
        result = result[:,:20000]


    regret =  np.mean(result,0)
    
    xcoords = np.arange(0,20000,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label ,  showlegend=False )) # 
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )
    # if label in [ 'CBP','RandCBP', 'TSPM_R1' ]:
    #     fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines', showlegend = False    ) ) 
    


# fig.show(legend=True)

fig.update_layout(autosize=False, #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1),
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                 font=dict(size=30,) )

fig.update_yaxes(range=[0, 30] )

# fig.write_image("./easy_AT.pdf")
# fig.update_xaxes( type="log" )
# fig.write_image("./easy_AT_log.pdf")

fig.show()

In [ ]:
n_cores = 16
n_folds = 100
horizon = 20000

task = 'hard'
game_type = 'AT'

game = games.apple_tasting(False)


colors = [   [250,0,0], [255,128,0], [255 , 0, 255], [0 , 0, 250], [0 , 128, 0] , [0,0,0] ] 
labels = [    'RandCBP', 'PM_DMEDc1',  'TSPM_R1', 'TSPM_R0', 'BPM_LEAST',  'CBP'  ] 

fig = go.Figure(    )

for color, label in list(zip(colors, labels))[::-1]:

    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    # np.save('./results/apple_tasting/easy_{}_{}_{}'.format(horizon,n_folds, label), result)

    if label == 'RandCBP':
        direct = './results/benchmark_randcbp/AT/'
        path = os.path.join(direct, 'balanced_20000_96_RandCBP_1_5.pkl.gz' )
        result = loadall_results(path, horizon, n_folds)

        # result = np.load( './results/benchmark_randcbp/{}/{}_{}_{}_Gaussian_1_10_10e7.npy'.format(game_type, task, horizon,n_folds) )
        # result = result[:,:20000]
        
    elif label == 'PM_DMEDc1':
        result = np.load('./results/AT/hard_{}_{}_{}.npy'.format(20000,16, label) )
    else: 
        result = np.load('./results/{}/{}_{}_{}_{}.npy'.format(game_type, task, 100000,n_folds, label) )
        result = result[:,:20000]


    regret =  np.mean(result,0) 
    
    xcoords = np.arange(0,20000,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    worst_case = np.argmax( result[:,-1] )

    upper_regret =  regret + std
    lower_regret =  regret - std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label ,  showlegend=False )) # 
    # if label in [ 'RandCBP', 'PM_DMEDc1', 'TSPM_R1' ]:
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )
        # fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=lower_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
        #                  line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )

    # if label in [ 'RandCBP', 'PM_DMEDc1', 'TSPM_R1' ]:
    #     fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines', showlegend = False    ) ) 
    

fig.update_layout( autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=25,) ) #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1),

fig.update_yaxes(range=[0, 250] )

# fig.write_image("./{}_{}.pdf".format(task, game_type))
# fig.update_xaxes( type="log" )
# fig.write_image("./{}_AT_log.pdf")
fig.show()

# fig.write_image("./hard_AT.pdf")

In [11]:
n_folds = 100
horizon = 100000

labels = [  'CBP', 'RandCBP'  ]  

final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/AT/hard_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/AT/hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] < final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( 'hard ',sum(final[:,0]) )


final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/AT/easy_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/AT/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] < final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( 'easy ',sum(final[:,0]) )

hard  74
easy  92


# Label Efficient:

In [ ]:


n_cores = 16
n_folds = 96
horizon = 20000

game = games.label_efficient(  )


colors = [   [250,0,0], [255,128,0],  [0,0,0] ] 
labels = [    'RandCBP', 'PM_DMEDc1',   'CBP'  ] 

fig = go.Figure( )

final_regrets = []

for color, label in list(zip(colors, labels))[::-1]:

    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/label_efficient/hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    
    if label == 'RandCBP':
        # result = np.load( './results/benchmark_randcbp/LE/LE_hard_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
        # result = result[:,:20000]

        direct = './results/benchmark_randcbp/LE/'
        path = os.path.join(direct, 'balanced_20000_96_RandCBP_1_5.pkl.gz' )
        result = loadall_results(path, horizon, n_folds)
        
    elif label == 'PM_DMEDc1':
        result = np.load('./results/LE/hard_{}_{}_{}.npy'.format(20000,16, label) )
    else: 
        result = np.load( './results/LE/hard_{}_{}_{}.npy'.format(100000,100, label) )
        result = result[:,:20000]

    final_regrets.append( result[:,-1] )
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,20000,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std


    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label,  showlegend=False )) # 
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )

    # fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines', showlegend = False    ) ) 

fig.update_layout(autosize=False,
                  xaxis_title="Sequence", yaxis_title="Regret",  margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),   font=dict(size=30,) )
                  #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1

fig.update_yaxes( type="log" )
# fig.write_image("./hard_LE.pdf")

# fig.update_xaxes( type="log" )
# fig.write_image("./hard_LE_log.pdf")

fig.show()
# fig.write_image("./hard_LE.pdf")

# final_regrets = np.array(final_regrets)
# final = [ ( np.argmin(final_regrets[:,i]), i) for i in range(n_folds) ]

In [ ]:


n_cores = 16
n_folds = 96
horizon = 20000

game = games.label_efficient(  )


colors = [  [255,128,0],  [250,0,0] ,  [0,0,0]] 
labels = [  'PM_DMEDc1',  'RandCBP',   'CBP'  ] 


fig = go.Figure( )

final_regrets = []


for color, label in list(zip(colors, labels))[::-1]:

    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/label_efficient/hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    
    if label == 'RandCBP':
        # result = np.load( './results/benchmark_randcbp/LE/LE_easy_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
        # result = result[:,:20000]
        direct = './results/benchmark_randcbp/LE/'
        path = os.path.join(direct, 'imbalanced_20000_96_RandCBP_1_5.pkl.gz' )
        result = loadall_results(path, horizon, n_folds)

    elif label == 'PM_DMEDc1':
        result = np.load('./results/LE/easy_{}_{}_{}.npy'.format(20000,16, label) )
    else: 
        result = np.load( './results/LE/easy_{}_{}_{}.npy'.format(100000,100, label) )
        result = result[:,:20000]

    final_regrets.append( result[:,-1] )
    
    regret =  np.mean(result,0) 
    print(regret[-1])
    xcoords = np.arange(0,20000,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    if label == 'PM_DMEDc1':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name='PMDMED (c=1)',  showlegend=False ))  

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label,  showlegend=False )) # 
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )
    # fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines', showlegend = False    ) ) 
    


fig.update_layout( autosize=False, xaxis_title="Sequence", yaxis_title="Regret",  
                   margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),   font=dict(size=30,) ) #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1),


fig.update_yaxes(  type="log" )# type="log",
# fig.write_image("./easy_LE.pdf")

# fig.update_xaxes( type="log" )
# fig.write_image("./easy_LE_log.pdf")

fig.show()

# final_regrets = np.array(final_regrets)
# final = [ ( np.argmin(final_regrets[:,i]), i) for i in range(n_folds) ]

In [ ]:
### HARD Label Efficienc:

n_cores = 16
n_folds = 100
horizon = 100000

labels = [  'CBP', 'RandCBP'  ]  

final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/LE/LE_hard_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/AT/hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] <= final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( 'hard', sum(final[:,0]) )


final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/LE/LE_easy_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/AT/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] < final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( 'easy', sum(final[:,0]) )